# Imports:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import IPython
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras import initializers, optimizers
from keras.callbacks import Callback
import kmapper as km
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from gtda.homology import VietorisRipsPersistence
from gtda.plotting import plot_diagram
import gudhi as gd

# Data

### MNIST podatkovni set  
- Vsebuje slike ročno napisanih številk velikosti **28×28 pik**  
- Za ta eksperiment smo podatke **filtrirali tako, da vključujejo le števki 3 in 5**

In [ ]:
# Naložimo podatkovni set
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Pomožna funkcija za filtriranje
def filter_digits(X, y, digits=(3, 5)):
    mask = np.isin(y, digits)
    return X[mask], y[mask]

# Filtriranje podatkov
X_train, y_train = filter_digits(X_train, y_train, digits=(3, 5))
X_test, y_test = filter_digits(X_test, y_test, digits=(3, 5))

# Primeri podatkov
fig = plt.figure(figsize=(15, 3))
rows, columns = 1, 5

# Izbremo 5 naključnih slik
random_indices = np.random.choice(len(X_train), size=5, replace=False)

for i, idx in enumerate(random_indices, 1):
    fig.add_subplot(rows, columns, i)
    img = X_train[idx]

    plt.imshow(img, cmap='gray')
    plt.title(f'Oznaka = {y_train[idx]}')
    plt.axis('off')

plt.tight_layout()

# Preprocessing

In [ ]:
# Normalizacija
X_train, X_test = X_train / 255.0, X_test / 255.0

# Pretvorba slik v vektorje pixlov
pixels = np.prod(X_train.shape[1:])
X_train = X_train.reshape(X_train.shape[0], pixels).astype('float32')
X_test = X_test.reshape(X_test.shape[0], pixels).astype('float32')
print('Train shape: {}\nTest shape: {}'.format(X_train.shape, X_test.shape))

# Ponovna označitev: 3 → 0, 5 → 1
y_train = (y_train == 5).astype(int)
y_test = (y_test == 5).astype(int)

# Kodiranje oznak v one-hot vektorje
y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

# Nevronska mreža

In [ ]:
def train_model(X_train, 
                y_train, 
                X_test, 
                y_test,
                neurons,
                init=initializers.RandomNormal(mean=0.0, stddev=0.01, seed=42),
                activation='sigmoid', 
                l1=0.0, 
                l2=0.0, 
                loss='categorical_crossentropy', 
                learning_rate=0.01,
                epochs=5, 
                batch_size=100, 
                training_steps=500):
    
    activ = [activation]*(len(neurons)-2)+['softmax']

    model = Sequential()
    for n in range(1, len(neurons)):
        model.add(Dense(neurons[n],
                    input_dim=neurons[n-1], 
                    kernel_initializer=init,
                    use_bias=False,
                    activation=activ[n-1]))

    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.SGD(learning_rate=learning_rate, momentum=0.0, decay=0.0, nesterov=False),
                  metrics=['accuracy'])

    # Train model
    training_steps = training_steps    # Desired number of training steps (approximate)
    N_ws = round((len(X_train) * epochs) / (batch_size * training_steps))
    WSaver = SaveWeights(N_ws)  # not SaveWeights(model, N_ws)


    calback_list = [WSaver]
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), 
                        epochs=epochs, batch_size=batch_size, callbacks=calback_list, verbose=2)
    scores = model.evaluate(X_test, y_test, verbose=0)

    W_layer = {}
    for n in range(len(model.layers)):
        W_layer[n] = WSaver.weights_layer[n]
    steps = len(W_layer[0])
    # Lists of vectors of weights for each neuron for each training step
    X_layer = {}
    for n in range(len(model.layers)):
        X_layer[n] = np.squeeze([W_layer[n][i][:, [j]] for i in range(steps) for j in range(neurons[n+1])])
    # Labels = [neuron number, training step]
    y_layer = {}
    for n in range(len(model.layers)):
        y_layer[n] = np.array([[j, i] for i in range(steps) for j in range(neurons[n+1])])
    # Number of weight matrices saved = 1 + (60000 * epochs) / (batch_size * N_ws)
    print('Training steps: {}'.format(steps))

    # Plot training and validation accuracy (and loss)
    plt.figure(figsize=(15, 4))
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.tight_layout()
    plt.show()
    
    return W_layer, X_layer, y_layer

# Callback function to save weights after each N mini-batches

class SaveWeights(Callback):
    def __init__(self, N):
        super().__init__()
        self.N = N
        self.batch = 0
        self.weights_layer = {}

    def set_model(self, model):
        super().set_model(model)  # ✅ This sets the read-only 'model' attribute

    def on_train_begin(self, logs=None):
        for n in range(len(self.model.layers)):
            self.weights_layer[n] = [self.model.layers[n].get_weights()[0]]

    def on_batch_end(self, batch, logs=None):
        if self.batch % self.N == 0:
            for n in range(len(self.model.layers)):
                self.weights_layer[n].append(self.model.layers[n].get_weights()[0])
        self.batch += 1


# Testiranje

In [ ]:
# Inicializiramo uteži
init = initializers.GlorotUniform(seed=42)

# Arhitektura nevronske mreže
neurons = [pixels, 100, 100, 2]

# Train 
W_layer, X_layer, y_layer = train_model(X_train, y_train, X_test, y_test,
                                        neurons=neurons,
                                        init=init,
                                        activation='sigmoid',
                                        loss='categorical_crossentropy',
                                        l1=0.0, l2=0.0, 
                                        learning_rate=0.05,
                                        epochs=70, 
                                        batch_size=100,
                                        training_steps=200)

### Vizualizacija uteži v prostoru PCA

Uteži iz vsake plasti nevronske mreže smo preslikali v dvodimenzionalni prostor s pomočjo **PCA (Principal Component Analysis)**, da bi lažje razumeli njihovo porazdelitev in evolucijo skozi učenje.

- **Vsaka točka predstavlja en nevron** ob določenem učnem koraku.
- Barva točke označuje bodisi:
  - **učni korak** (leva slika), ali
  - **identiteto nevrona** (desna slika).

#### V zadnjem sloju:
- **Nevron 0** predstavlja slike števila **3**
- **Nevron 1** predstavlja slike števila **5**


In [ ]:
for lay_num in range(len(X_layer)):
    X_pca2 = PCA(n_components=2).fit_transform(X_layer[lay_num])
    
    plt.figure(figsize=(10, 4))
    
    # colored by training step
    plt.subplot(121)
    scatter1 = plt.scatter(X_pca2[:, 0], X_pca2[:, 1], s=3, c=y_layer[lay_num][:, 1],
                           cmap=plt.cm.get_cmap('viridis'), alpha=0.8)
    plt.title(f'Plast {lay_num} (obarvano po učnem koraku)')
    plt.colorbar(scatter1, label="Učni korak")

    # colored by neuron ID
    plt.subplot(122)
    scatter2 = plt.scatter(X_pca2[:, 0], X_pca2[:, 1], s=4, c=y_layer[lay_num][:, 0],
                           cmap=plt.cm.get_cmap('jet'), alpha=0.8)
    plt.title(f'Plast {lay_num} (obarvano po nevronu)')

    # Add colorbar with neuron labels
    cbar = plt.colorbar(scatter2)
    cbar.set_label("ID nevrona")

    # Diskretne labele nevronov prikažemo samo v zadnjem koraku
    if lay_num == len(X_layer) - 1:
        neuron_ids = np.unique(y_layer[lay_num][:, 0])
        cbar.set_ticks(neuron_ids)
        cbar.set_ticklabels([f"Nevron {int(n)}" for n in neuron_ids])

    plt.tight_layout()
    plt.show()


### Mapper

In [ ]:
def graph_km(data, 
             label, 
             path, 
             projection=PCA(3), 
             title='Title', 
             color_function='Step', 
             nr_cubes=45, 
             overlap_perc=0.09, 
             clusterer=DBSCAN(eps=0.3, min_samples=15)):
    
    # Initialize KeplerMapper (used for lens only)
    mapper = km.KeplerMapper(verbose=1)

    # Create the projection (lens)
    lens = mapper.fit_transform(data, projection=projection, scaler=None)

    # Create the simplicial complex
    graph = mapper.map(lens,
                data,
                cover=km.Cover(nr_cubes, overlap_perc),  
                clusterer=clusterer)
    
    # Color values function
    if color_function == 'Class': 
        color_func = label[:,0]
        color_func_name = "Obarvano po nevronu"
    elif color_function == 'Step':
        color_func = label[:,1]
        color_func_name = "Obarvano po učnem koraku"


    html = mapper.visualize(graph,
                            path_html=path,
                            title=title,
                            color_values=color_func,
                            color_function_name=color_func_name)

Brez zmanjševanja dimenzij

PCA

In [ ]:
lay_num = 0
proj = PCA(2)
graph_km(X_layer[lay_num], y_layer[lay_num], 
         projection=proj, 
         title='Weights to layer',
         path='output/PCA_Graph_{}.html'.format(lay_num))
IPython.display.IFrame('output/PCA_Graph_{}.html'.format(lay_num), 800, 600)

In [ ]:
lay_num = 1
proj = PCA(2)
graph_km(X_layer[lay_num], y_layer[lay_num], 
         projection=proj, 
         title='Weights to layer',
         path='output/PCA_Graph_{}.html'.format(lay_num))
IPython.display.IFrame('PCA_Graph_{}.html'.format(lay_num), 800, 600)

In [ ]:
lay_num = 2
proj = PCA(2)
graph_km(X_layer[lay_num], y_layer[lay_num], 
         projection=proj, 
         title='Weights to layer',
         path='output/PCA_Graph_{}.html'.format(lay_num))
IPython.display.IFrame('output/PCA_Graph_{}.html'.format(lay_num), 800, 600)

TSNE

In [ ]:
lay_num = 0
proj = TSNE(n_components=2)
graph_km(X_layer[lay_num], y_layer[lay_num], 
         projection=proj, 
         title='Weights to layer',
         path='output/TSNE_Graph_{}.html'.format(lay_num))
IPython.display.IFrame('TSNE_Graph_{}.html'.format(lay_num), 800, 600)

In [ ]:
lay_num = 1
proj = TSNE(n_components=2)
graph_km(X_layer[lay_num], y_layer[lay_num], 
         projection=proj, 
         title='Weights to layer',
         path='output/TSNE_Graph_{}.html'.format(lay_num))
IPython.display.IFrame('TSNE_Graph_{}.html'.format(lay_num), 800, 600)

In [ ]:
lay_num = 2
proj = TSNE(n_components=2)
graph_km(X_layer[lay_num], y_layer[lay_num], 
         projection=proj, 
         title='Weights to layer',
         path='output/TSNE_Graph_{}.html'.format(lay_num),
         clusterer=DBSCAN(eps=1.0, min_samples=2))
IPython.display.IFrame('TSNE_Graph_{}.html'.format(lay_num), 800, 600)

L2 norm

In [ ]:
lay_num = 0
proj = 'l2norm'
graph_km(X_layer[lay_num], y_layer[lay_num], 
         projection=proj, 
         title='Weights to layer',
         path='output/L2_Graph_{}.html'.format(lay_num))
IPython.display.IFrame('L2_Graph_{}.html'.format(lay_num), 800, 600)

In [ ]:
lay_num = 1
proj = 'l2norm'
graph_km(X_layer[lay_num], y_layer[lay_num], 
         projection=proj, 
         title='Weights to layer',
         path='output/L2_Graph_{}.html'.format(lay_num))
IPython.display.IFrame('L2_Graph_{}.html'.format(lay_num), 800, 600)

In [ ]:
lay_num = 2
proj = 'l2norm'
graph_km(X_layer[lay_num], y_layer[lay_num], 
         projection=proj, 
         title='Weights to layer',
         path='output/L2_Graph_{}.html'.format(lay_num))
IPython.display.IFrame('L2_Graph_{}.html'.format(lay_num), 800, 600)

In [ ]:
lay_num = 2
X = X_layer[lay_num] 

# Izračunamo vztrajno homologijo
vr = VietorisRipsPersistence(homology_dimensions=[0, 1])  # H0, H1
diagrams = vr.fit_transform([X])

plot_diagram(diagrams[0])

In [ ]:
X = X_layer[2]

# Create the RipsComplex
rips_complex = gd.RipsComplex(points=X, max_edge_length=2.0)

# Create a simplex tree (filtration)
simplex_tree = rips_complex.create_simplex_tree(max_dimension=2)

# Compute persistence
diag = simplex_tree.persistence(homology_coeff_field=2, min_persistence=0.01)

In [ ]:
# Plot persistence diagram
gd.plot_persistence_diagram(diag)
plt.title("Persistence Diagram")

# Plot barcode
gd.plot_persistence_barcode(diag)
plt.title("Persistence Barcode")